<a href="https://colab.research.google.com/github/sugandhab31/NotesRAGDemoApp/blob/main/NotesRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install -U requests==2.32.5 --quiet
!pip install -U "langgraph>=0.2.26"  --quiet
!pip install langchain langchain-community sentence-transformers faiss-cpu transformers accelerate torch --quiet
!python -m pip show langgraph


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.4 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.2 requires langchain-core>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
Name: langgraph
Version: 1.0.2
Summary: Building stateful, multi-actor applications with LLMs
Home-page: 
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-prebuilt, langgraph-sdk, pydantic, xxhash
Required-by: 


In [11]:
from typing import TypedDict
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from datetime import datetime
import pandas as pd



notes_db = [
    {"id": 1, "tags": ["AI", "LangGraph"], "text": "LangGraph lets you build agent workflows as graphs with memory and control flow."},
    {"id": 2, "tags": ["AI"], "text": "Retrieval-Augmented Generation (RAG) connects LLMs to external knowledge to reduce hallucinations."},
    {"id": 3, "tags": ["Python"], "text": "FastAPI is a fast, modern framework for building web APIs in Python."},
    {"id": 4, "tags": ["AI", "Embeddings"], "text": "Sentence-Transformers provide high-quality text embeddings for semantic search."}
]


In [12]:
# Initialize embedding model wrapper for LangChain
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

texts = [n["text"] for n in notes_db]
metas = [{"tags": n["tags"], "id": n["id"]} for n in notes_db]

# Build FAISS vector store (LangChain handles embeddings internally)
vectorstore = FAISS.from_texts(texts, embedding=embedding_model, metadatas=metas)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

/tmp/ipython-input-139050481.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
from langgraph.graph import StateGraph
from typing import TypedDict

class RAGState(TypedDict):
    query: str
    context: str
    answer: str

builder = StateGraph(state_schema=RAGState)

def retriever_node(state):
    docs = retriever.get_relevant_documents(state["query"])
    state["context"] = "\n".join([d.page_content for d in docs])
    return state

def summarizer_node(state):
    prompt = f"Answer concisely using only this context:\n{state['context']}\n\nQuestion: {state['query']}\nAnswer:"
    response = llm(prompt, max_new_tokens=150)[0]["generated_text"]
    state["answer"] = response
    return state

builder.add_node("retriever", retriever_node)
builder.add_node("summarizer", summarizer_node)
builder.add_edge("retriever", "summarizer")
builder.set_entry_point("retriever")
builder.set_finish_point("summarizer")

checkpointer = MemorySaver()

app = builder.compile(checkpointer=checkpointer)


In [27]:
from datetime import datetime
import pandas as pd

metrics = []

def run_query(query, session_id="default"):
  """Run a quey through RAG graph with session memory + metrics tracking"""
  config = {"configurable":{"thread_id": session_id}}
  state = app.invoke({"query": query, "context": "", "answer": ""}, config=config)

  # Print human-readable response
  print(f"\n🧠 Session: {session_id}")
  print(f"❓ Query: {query}")
  print(f"💬 Answer: {state['answer'][:250]}\n")

  completeness = round(min(1.0, len(state["context"].split()) / (len(query.split()) * 5)), 2)
  faithfulness = 1.0 if any(w in state["answer"].lower() for w in query.lower().split()[:2]) else 0.8
  mismatch = 1-faithfulness

  metrics.append({
      "session": session_id,
      "query": query,
      "completeness": completeness,
      "faithfulness": faithfulness,
      "mismatch": mismatch,
      "timestamp": datetime.now().isoformat()
  })

  return state


In [28]:
run_query("What is LangGraph?", "user001")
run_query("How does RAG reduce hallucinations?", "user001")
run_query("Explain FastAPI briefly", "user001")



🧠 Session: user001
❓ Query: What is LangGraph?
💬 Answer: LangGraph lets you build agent workflows as graphs with memory and control flow. Sentence-Transformers provide high-quality text embeddings for semantic search


🧠 Session: user001
❓ Query: How does RAG reduce hallucinations?
💬 Answer: connects LLMs to external knowledge


🧠 Session: user001
❓ Query: Explain FastAPI briefly
💬 Answer: FastAPI is a fast, modern framework for building web APIs in Python. LangGraph lets you build agent workflows as graphs with memory and control flow.



{'query': 'Explain FastAPI briefly',
 'context': 'FastAPI is a fast, modern framework for building web APIs in Python.\nLangGraph lets you build agent workflows as graphs with memory and control flow.',
 'answer': 'FastAPI is a fast, modern framework for building web APIs in Python. LangGraph lets you build agent workflows as graphs with memory and control flow.'}